# 이미지에서 고양이와 강아지를 분류하자!

필요한 라이브러리를 import합니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator   # Tensorflow에서 이미지를 전처리하기 위해 모아둔 기능이 들어있다.

import os  # 파일과 디렉토리 구조를 읽는데 사용
import numpy as np #  python list를 numpy array로 변환하고 필요한 matrix 연산을 수행한다.
import matplotlib.pyplot as plt # 그래프를 작성하고 training 및 validation data에 대한 이미지를 프리뷰

캐글에서 <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a> 데이터셋을 내려받아 준비합니다.

데이터셋의 구조는 아래와 같습니다.

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

얼마나 많은 고양이와 개의 이미지가 있는지 살펴봅시다.

In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

이미지를 전처리합니다

1. 디스크에서 이미지 읽기
2. 이미지의 픽셀값을 0-255에서 0-1로 조정합니다.

이 모든 작업은 `tf.keras`가 제공하는 `ImageDataGenerator`를 통해 수행할 수 있습니다.

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

디스크에서 이미지를 매번 미니배치 크기만큼 로드하고, 필요한 치수로 이미지 크기를 조정합니다.

강아지와 고양이 사진의 다양한 해상도가 있을 수 있기 때문에, 일괄 가로/세로 150픽셀씩이 되도록 바꿔주겠습니다.

In [ ]:
BATCH = 128
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')  

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

강아지와 고양이 이미지가 어떤 것들이 있는지 불러와 확인해봅니다.

디렉토리에서 랜덤하게 사진 다섯장을 불러옵니다. 아래 출력된 것은 라벨로, 고양이의 경우 0, 강아지의 경우 1임을 알 수 있습니다.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
sample_training_images, sample_training_labels = next(train_data_gen)
plotImages(sample_training_images[:5])
print(sample_training_labels[:5])

이미지 한 장당 가로세로 150픽셀 RGB컬러 이미지임을 알 수 있습니다.

In [ ]:
sample_training_images[0].shape  

# 실습 MISSION #1
* 작성된 네트워크 아키텍처 뒤에 ReLU Activation Function을 적용한 512 Dense Layer를 추가해보자.

* 그 뒤 output layer에서 2진분류가 가능하도록 1개 노드를 가진 Dense layer를 추가한다(0,1분류). 이 때 sigmoid를 활용한다.

힌트! 3차원 텐서가 흐르던 Conv 연산을 Dense로 다시 연결하려면 납작하게 1차원으로 눌러주는 작업이 필요하다..!


In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPool2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPool2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPool2D(),
    #### ANSWER ####
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
    ################
])

model.summary()

loss와 update 기법을 설정해준 뒤 학습을 진행합니다.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
EPOCHS = 15
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // BATCH,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=total_val // BATCH
)

학습이 끝났으면 Learning curve를 찍어봅니다.

위 설정대로 잘 학습하였다면, 그래프를 통해 Overfitting 문제가 발생한 것을 확인해볼 수 있습니다.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

어제 배운 Regularization 기법 중 Data augmentation을 적용해봅시다.

`ImageDataGenerator`를 사용하여 데이터셋에 다양한 변환을 줄 수 있습니다.

**(1) horizontal flip 적용**

ImageDataGenerator에서 horizontal_filp 옵션을 이용하여 좌우 반전을 줄 수 있습니다.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

교육 예제에서 하나의 샘플 이미지를 추출한 후 5회 반복하여 동일한 이미지에 5회 augmentation을 적용해보고 결과를 확인합니다.

이미지를 불러올 때마다 좌우 반전이 랜덤하게 적용된 것을 확인하실 수 있습니다.

마치 강아지와 고양이 사진을 이쪽에서도, 저쪽에서도 찍은 것 같은 효과를 줍니다.

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=BATCH,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

**(2) Randomly rotate the image**

rotation_range 옵션을 적용하여 학습 이미지가 무작위로 회전되게 할수 있습니다.

카메라를 돌려가며 여러 장 촬영한 것 같은 효과를 줍니다.

옵션값에는 회전을 허용할 각도 범위를 설정합니다.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=BATCH,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

**(3) zoom augmentation 적용**

zoom_range 옵션으로 이미지에 무작위로 범위 내 당겨찍기, 멀리찍기 효과를 줄 수 있습니다.

옵션값에는 줌을 허용할 범위를 0~1사이의 값으로 지정해줍니다.

0.5로 설정하면 50%-150% 범위로 당겨찍고 멀리찍은 효과를 줍니다.

좌우와 상하 각각 랜덤하게 적용됩니다

뚱뚱한 강아지(고양이), 길쭉한 강아지(고양이)가 다양하게 있는듯한 효과를 볼 수 있습니다.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5)  # 50% ~ 150%

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=BATCH,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

# 실습 MISSION #2 : augmentation 효과 한 번에 적용하기

* 픽셀값이 0~1 범위가 되도록 rescale
* horizontal flip(좌우반전) 허용
* 좌우 45도 범위 내 회전 허용
* zoom 확대/축소 10% 범위 내 허용
* width shift 중앙 기준 20%범위 허용
* height shift 중앙 기준 25%범위 허용

(힌트 : 맨 아래 두 기능은 알려드리지 않았지만, ImageDataGenerator 뒤에 마우스를 클릭하고 기다리면 뜨는 팝업을 통해 어떤 옵션명을 적용하면 좋을지 참고할 수 있습니다.)

[참고 Documentation](https://keras.io/preprocessing/image/)

In [ ]:
image_gen_train = ImageDataGenerator(
                    #### ANSWER #### 
                    rescale=1/255.,
                    horizontal_flip=True,
                    rotation_range=45,
                    zoom_range=0.1,
                    width_shift_range=0.2,
                    height_shift_range=0.25
                    ################
                    )

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)


data augmentation은 일반적으로 training set에만 적용합니다.

validation set은 `rescale`만 적용하겠습니다.

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

# 실습 MISSION #3 

Overfitting을 피하기 위해 Dropout을 추가하자

* 모든 Dense 레이어 앞부분에 30%만큼의 dropout 비율을 적용하자

In [ ]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPool2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPool2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPool2D(),
    Flatten(),
    ##### 미션을 작성하세요 ######
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
    ##############################
])
model_new.summary()


미션을 작성하였다면 50 epoch동안 네트워크를 학습시킵니다.

In [ ]:
model_new.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

epochs_new = 50

history = model_new.fit(
    train_data_gen,
    steps_per_epoch=total_train // BATCH,
    epochs=epochs_new,
    validation_data=val_data_gen,
    validation_steps=total_val // BATCH
)

학습이 끝난 후 Learning curve를 보면, overfitting이 크게 개선된 것을 확인할 수 있습니다. 

정확성을 높이기 위해서는 모델을 더 충분히 훈련시켜도 될 듯 해 보이네요.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs_range = range(epochs_new)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

학습한 모델을 현장에 적용해보겠습니다.

아래는 제가 인터넷에서 검색한 개와 고양이 이미지입니다.

한 번 아래 내용을 실행해보시고, 원하시는 아무 강아지나 고양이 이미지 url을 해당 위치에 넣고 직접 추론해보세요!


In [ ]:
!wget -O dog_sample.jpg https://www.guidingeyes.org/wp-content/uploads/2020/01/1-1.jpg
!wget -O cat_sample.jpg https://www.rd.com/wp-content/uploads/2019/11/cat-10-e1573844975155-768x519.jpg

In [ ]:
def img2input_keras(path, target_size):
  img_show = plt.imread(path)
  plt.imshow(img_show)
  tmp = tf.keras.preprocessing.image.load_img(path, target_size=target_size)
  tmp = tf.keras.preprocessing.image.img_to_array(tmp)
  tmp = tmp/255.
  tmp = np.expand_dims(tmp, axis=0)
  print(tmp.shape)
  return tmp

제가 가져온 이미지는 이렇게 생겼습니다.

In [ ]:
dog_input = img2input_keras('dog_sample.jpg', (150, 150))

In [ ]:
cat_input = img2input_keras('cat_sample.jpg', (150, 150))

1에 가까울수록 강아지, 0에 가까울수록 고양이라고 모델이 추론했다고 볼 수 있습니다.

**첫 번째 모델(overfitting)의 추론 결과**

In [ ]:
print('dog image -> ', model.predict(dog_input))
print('cat image -> ',model.predict(cat_input))

**두 번째 모델(augmentation 및 dropout 적용)의 추론 결과**

In [ ]:
print('with new model, dog image -> ', model_new.predict(dog_input))
print('with new model, cat image -> ', model_new.predict(cat_input))

데이터셋이 부족한 편이라 사람에 따라서는 두 모델 모두 추론 결과가 좋지 않을 수도 있습니다.

그리고 학습 데이터도, 외국의 강아지와 고양이 사진을 활용했기 때문에 조금 더 미국스러운(?) 사진을 널었을 때 추론을 더 잘하는 것처럼 보이기도 합니다.

구글에서 다양한 사진을 검색해서 넣어보고, 두 모델의 추론 결과를 비교해봅시다 😊

In [ ]:
!wget -O test_sample.jpg https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/article_thumbnails/other/dog_cool_summer_slideshow/1800x1200_dog_cool_summer_other.jpg

test_input = img2input_keras('test_sample.jpg', (150, 150))
print('with new model, test image -> ', model_new.predict(test_input))